In [101]:
%load_ext autoreload
%autoreload 2

import time

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")
sheet = wb.sheets['Sheet1']

#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]


instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs]
    #compile_excel(wb.sheets["Pre-Check BUY"],cells[(instruments.v_model, 'Buy')])
    

instruments.on_event('change',select_change)

widgets = {}
cells = {}
from copy import deepcopy
for item in interface:
    item = deepcopy(item)
    instrument, type_, obj = item.pop('INSTRUMENT'), item.pop('TYPE'), item.pop('OBJECT')
    widgets.setdefault((instrument, type_), [])
    cells.setdefault((instrument, type_), [])
    label = item['label'].replace(' ','_')
    widget = getattr(v, obj)(**item)
    widget.id = f"{instrument}_{type_}_{label}"
    widgets[instrument, type_].append(widget)
    cells[instrument, type_].append((label,item['cell']))


#forms    
forms = {}
for k, children in widgets.items():
        forms[k] = v.Form(children=children)

class AllForms:
    def __init__(self):
        for k, form in forms.items():
            for widget in form.children:
                setattr(self, widget.id, widget)

af = AllForms()

#Buttons Form
Proceed = v.Btn(
                  icon = False,
                  children=[
                      v.Icon(children=['mdi-application-export'],  class_ = "mr-2"),
                      'Proceed',
                      ])

def compile_excel(sheet_sell,cells):
    for f in cells:
        obj = getattr(af,f"{instruments.v_model}_Buy_{f[0]}")
        obj.v_model = sheet_sell.range(f[1])
        print(obj)
        
        
#functions to be called
def Test(sheet_sell):
    sheet_sell.range("C19").value= ticker.v_model  
    if sheet_sell.range("C27").value == "#N/A Requesting Data..." or sheet_sell.range("C30").value == "#N/A Requesting Data...":
        time.sleep(0.1)
        Test(sheet_sell)
    else:
        #print(sheet_sell.range("C27").value)
        Px_Last.v_model = sheet_sell.range("C27").value
        ISIN.v_model = sheet_sell.range("B20").value
        Name.v_model = sheet_sell.range("C20").value
        Crncy.v_model = sheet_sell.range("C28").value
        Cross.v_model = sheet_sell.range("C29").value
        App_Cross.v_model = sheet_sell.range("C30").value
        Price_Euro.v_model = sheet_sell.range("C31").value
        #print(st.range("A5").value)

    #ticker.v_model = sheet_sell.range("C25").value

    
def verify(widget, event, data):
    Proceed.disabled = False

Proceed.on_event('click',Test)

   
#form= v.Form(children=[ticker,widgets.HBox([ISIN, Name],style_='width: 100%; opacity: 0.7'),
#                       Px_Last,Crncy,Cross,App_Cross,Price_Euro,Quantity,Verify, Proceed])


#tabs
content_list = [v.TabItem(children=[forms[('Equity','Buy')]]),v.TabItem(children=[forms[('Equity','Sell')]])]
tab_list = [v.Tab(children=["Pre-Trade Buy"]), v.Tab(children=["Pre-Check Sell"])]

tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")

###############
#APP
###############

#navigation bar
navDrawer = v.NavigationDrawer(v_model=True, children=[
     v.Select(label='select1', items=[1,2,], prepend_icon = 'fa-truck'), 
    v.TextField(label='text', prepend_icon = 'mdi-heart'),
    v.Select(label='select3', prepend_icon = 'mdi-magnify')
])

navDrawer.mini_variant = True
navDrawer.expand_on_hover = True

#navDrawer.width =300
navDrawer.mini_variant_width = 30
navDrawer.mini_variant_width = 0

toolBarButton = v.Btn(
                  icon = True, 
                  children=[
                      v.Icon(children=['mdi-dots-vertical'])
                      ])

def on_click(widget, event, data):
    navDrawer.v_model = not navDrawer.v_model
    if navDrawer.v_model:
        navDrawer.mini_variant_width = 30
    else:
        navDrawer.mini_variant_width = 0
        
#toolBarButton.on_event('click', on_click)

#Init
v.theme.dark = True
app = v.App(v_model=1)

appBar = v.AppBar(
      #color="light-blue darken-3",
    color="blue-grey darken-1",
      dense=True,
      dark = True   ,  
       children = [
           toolBarButton, 
           v.ToolbarTitle(children=['Niche IMS']),
                    ])

content = v.Col(children = [instruments,tabs,Proceed])


drawersWithContent = v.Container(
        class_="fill-height",
         fluid=True,
        children = [
                     v.Row(
                          align="top",
                      justify="left",
                        children = [navDrawer,content]
                            )
                ])

app.children = [appBar,drawersWithContent]


app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


App(children=[AppBar(children=[Btn(children=[Icon(children=['mdi-dots-vertical'])], icon=True), ToolbarTitle(c…

In [104]:
def compile_excel(sheet_sell,cells):
    for f in cells:
        obj = getattr(af,f"{instruments.v_model}_Buy_{f[0]}")
        print(f[1])
        obj.v_model = sheet_sell.range(f[1]).value
        print(obj)

In [109]:
instruments.v_model
sheet=wb.sheets["Pre-Check BUY"]
print(sheet.name)
cells[(instruments.v_model, 'Buy')][0:2]

Pre-Check BUY


[('Ticker_Bloomberg', 'C20'), ('ISIN', 'B21')]

In [52]:
for c in cells[(instruments.v_model, 'Buy')][0:2]:
    print(c)

('ISIN', 'C90')
('Px_Last', 'C93')


In [73]:
af.Bond_Buy_ISIN.v_model ='3'

In [110]:
b=getattr(af,'Bond_Buy_ISIN')
print(sheet.range("C90").value)
b.v_model = sheet.range("C90").value

US345370CW84


In [111]:
args2 = cells[(instruments.v_model, 'Buy')]
compile_excel(sheet,args2)

C20
TextField(disabled=False, filled=False, id='Equity_Buy_Ticker_Bloomberg', label='Ticker Bloomberg', v_model='TEF SM')
B21
TextField(disabled=True, filled=True, id='Equity_Buy_ISIN', label='ISIN', v_model=None)
C21
TextField(disabled=True, filled=True, id='Equity_Buy_Name', label='Name', v_model=None)
C22
TextField(disabled=True, filled=True, id='Equity_Buy_Px_Last', label='Px Last', v_model=None)
C23
TextField(disabled=True, filled=True, id='Equity_Buy_Currency', label='Currency', v_model=None)
C24
TextField(disabled=True, filled=True, id='Equity_Buy_Cross', label='Cross', v_model=None)
C25
TextField(disabled=True, filled=True, id='Equity_Buy_App.cross', label='App.cross', v_model=None)
C26
TextField(disabled=True, filled=True, id='Equity_Buy_Price_Euro', label='Price Euro', v_model=None)
C29
TextField(disabled=False, filled=False, id='Equity_Buy_Quantity', label='Quantity', v_model=1293.0)
C32
TextField(disabled=True, filled=True, id='Equity_Buy_Cash-out_euro', label='Cash-out eur

In [92]:
1/3

0.3333333333333333

In [93]:
1/7

0.14285714285714285

In [113]:
sheet.range('C73').value

0.2